In [1]:
import locale
   ...: locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

import pandas as pd
import pyarrow
import plotly.express as px
import yaml
from IPython.display import display, Markdown
import datetime
import dateutil.relativedelta
import sys

#for key, value in locale.localeconv().items():
#    print("%s: %s" % (key, value))

## Acompanhamento das pessoas com Tuberculose

Essa seção se dedica a identificar como é o registro do acompanhamento de pessoas com Tuberculose no município.

As pessoas foram computadas se tivessem códigos CID-10, CIAP-2 ou ABP relacionados ao diagnóstico de Tuberculose registrados, e avaliou-se se o acompanhamento por atendimentos, basciloscopia e visitas domiciliares está adequada.

In [2]:

display(Markdown("## Dados gerais de perfil do indicador"))

with open('P11_Tuberculose.yml', 'r') as file:
    indicador = yaml.safe_load(file)

for P in indicador['perfil'].keys():
    display(Markdown(f"{P}: {indicador['perfil'][P]}  "))
#    print(P + ": " + str(indicador['perfil'][P]) + " ")


## Dados gerais de perfil do indicador

Painel_version: 1.0.5  

nome_indicador: Tuberculose  

codigo_indicador: P11  

data: 2024-11-28  

fichas: ['tb_fat_atend_individual', 'tb_fat_visita_domiciliar']  

tipo: cobertura proporcional  

meses: 4  

In [3]:


df = pd.read_parquet('Cidade_X/atendimentos.parquet')

display(Markdown("## Dados gerais da população atendida"))

display(Markdown("### Todo período do banco"))

# Nota: preciso delimitar os últimos 12 meses apenas.

atendimentos_total = df.shape[0]
populacao_total = df['paciente_id'].nunique()

# Deixando o número em português

pt_atendimentos = locale.format_string('%u', atendimentos_total, grouping=True )
pt_populacao = locale.format_string('%u', populacao_total, grouping=True )

display(Markdown(f"Total de atendimentos = **{pt_atendimentos}**.  "))
display(Markdown(f"Pessoas atendidas = **{pt_populacao}**.  "))

agora = pd.Timestamp.now()
antiga = df.co_dim_tempo.min()
recente = df.co_dim_tempo.max()

display(Markdown("**Atualização do banco**  "))
display(Markdown(f"Data e horas de hoje: {agora.strftime('%Y-%m-%d')}  "))
display(Markdown(f"Registro mais antigo: {antiga}  "))
display(Markdown(f"Registro mais recente: {recente}  "))

display(Markdown(f"**Período para apuração do indicador:**  {indicador['perfil']['meses']} meses"))

inicio = agora - dateutil.relativedelta.relativedelta(months=int(indicador['perfil']['meses']))


## Dados gerais da população atendida

### Todo período do banco

Total de atendimentos = **837.928**.  

Pessoas atendidas = **81.357**.  

**Atualização do banco**  

Data e horas de hoje: 2024-01-29  

Registro mais antigo: 2011-01-01  

Registro mais recente: 2024-01-21  

**Período para apuração do indicador:**  4 meses

In [4]:
# display(Markdown("### Período de apuração do indicador"))

display(Markdown(f"Serão considerados os atendimentos a partir de **{inicio.strftime('%Y-%m-%d')}**"))

df['co_dim_tempo'] = pd.to_datetime(df['co_dim_tempo'])
df = df.loc[(df['co_dim_tempo'] >= inicio)]

atendimentos_total = df.shape[0]
populacao_total = df['paciente_id'].nunique()

# Deixando o número em português

pt_atendimentos = locale.format_string('%u', atendimentos_total, grouping=True )
pt_populacao = locale.format_string('%u', populacao_total, grouping=True )

display(Markdown(f"Total de atendimentos em {indicador['perfil']['meses']} meses = **{pt_atendimentos}**.  "))
display(Markdown(f"Pessoas atendidas {indicador['perfil']['meses']} meses= **{pt_populacao}**.  "))


Serão considerados os atendimentos a partir de **2023-09-29**

Total de atendimentos em 4 meses = **40.241**.  

Pessoas atendidas 4 meses= **19.628**.  

In [5]:
# df.dtypes

In [6]:


display(Markdown("## Cálculo do denominador"))

display(Markdown("### Atendimentos codificados com CID-10"))

df1 = pd.DataFrame()
df2 = pd.DataFrame()

for C in indicador['denominador']['filtro_cids']:
    df1 = df[df['ds_filtro_cids'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f"CID {C} encontrados **{N_atend}**  "))
    df2 = pd.concat([df1, df2], ignore_index=True).drop_duplicates()

atd_cid = df2.shape[0]
pessoas_cid = df2['paciente_id'].nunique()

display(Markdown(f"Atendimentos por {indicador['perfil']['nome_indicador']}, removendo duplicados = **{atd_cid}**.  "))
display(Markdown(f"Pessoas com {indicador['perfil']['nome_indicador']} encontradas via CID-10 = **{pessoas_cid}**.  "))

## Cálculo do denominador

### Atendimentos codificados com CID-10

CID A15 encontrados **18**  

CID A16 encontrados **2**  

CID A17 encontrados **0**  

...

CID A19 encontrados **0**  

CID B90 encontrados **1**  

CID J65 encontrados **0**  

Atendimentos por Tuberculose, removendo duplicados = **21**.  

Pessoas com Tuberculose encontradas via CID-10 = **8**.  

In [7]:


display(Markdown("### Atendimentos codificados com CIAP-2 ou ABP"))

df1 = pd.DataFrame()
df3 = pd.DataFrame()


for C in indicador['denominador']['filtro_ciaps']:
    df1 = df[df['ds_filtro_ciaps'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f" {C} encontrados **{N_atend}**  "))
    df3 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

atd_ciap = df3.shape[0]
pessoas_ciap = df3['paciente_id'].nunique()

display(Markdown(f"Atendimentos por {indicador['perfil']['nome_indicador']}, removendo duplicados = **{atd_ciap}**."))
display(Markdown(f"Pessoas com {indicador['perfil']['nome_indicador']} encontradas via CID-10 = **{pessoas_ciap}**."))

### Atendimentos codificados com CIAP-2 ou ABP

 A70 encontrados **22**  

 ABP017 encontrados **0**  

Atendimentos por Tuberculose, removendo duplicados = **22**.

Pessoas com Tuberculose encontradas via CID-10 = **10**.

In [8]:


display(Markdown(f"### Total de pessoas para: {indicador['perfil']['nome_indicador']}"))

lista_denominador = pd.concat([df2, df3], ignore_index=True).drop_duplicates()

atd_alvo = lista_denominador.shape[0]
populacao_alvo = lista_denominador['paciente_id'].nunique()

display(Markdown(f"Total de atendimentos pela condição alvo = **{atd_alvo}**.  "))
display(Markdown(f"Total de pessoas na população alvo = **{populacao_alvo}**."))
    

### Total de pessoas para: Tuberculose

Total de atendimentos pela condição alvo = **34**.  

Total de pessoas na população alvo = **11**.

In [9]:


import plotly.graph_objects as go

labels = ['População alvo','População total']
values = [populacao_alvo, populacao_total]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['blue', 'cyan']))
fig.show()

percentual = populacao_alvo / populacao_total * 100

percentual = locale.format_string('%10.2f', percentual)

display(Markdown(f"Sua população-alvo para esse indicador corresponde a {percentual}% da população total."))

if populacao_alvo == 0:
    print("Não foi encontrada população-alvo. Não será possível calcular indicador proporcional")


Sua população-alvo para esse indicador corresponde a       0,06% da população total.

## Cálculo do Numerador

In [10]:


display(Markdown(f"### Atendimentos com abordagem individual de {indicador['perfil']['nome_indicador']}"))

df0 = lista_denominador
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

display(Markdown("### Perfil da Condição A"))

for P in indicador['condição_A'].keys():
    display(Markdown(f"{P}: {indicador['condição_A'][P]}  "))


### Atendimentos com abordagem individual de Tuberculose

### Perfil da Condição A

descrição: ao menos 4 consultas no quadrimestre  

fonte: tb_fat_atend_individual  

profissional: ['medico', 'enfermeiro']  

valor_min: 4  

In [11]:
display(Markdown("### Avaliação da Condição A:"))

# Conta todas os atendimentos recebidos

atendimentos_alvo = df0.shape[0]
pessoas_beneficiadas = df0
df_pacientes_atendimentos = df0.groupby(['paciente_id']).agg(co_dim_tempo_count=('co_dim_tempo', 'count')).reset_index()
pessoas_beneficiadas = df0['paciente_id'].nunique()

df_condição_A = df_pacientes_atendimentos[df_pacientes_atendimentos.co_dim_tempo_count > indicador['condição_A']['valor_min']]

pessoas_beneficiadas_A = df_condição_A.shape[0]

display(Markdown(f"Total de atendimentos com avaliação de {indicador['perfil']['nome_indicador']} = **{atendimentos_alvo}**.  "))
display(Markdown(f"Pessoas beneficiadas com ao menos 1 atendimento = **{pessoas_beneficiadas}**."))
display(Markdown(f"Pessoas beneficiadas com ao menos {indicador['condição_A']['valor_min']} atendimentos = **{pessoas_beneficiadas_A}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Avaliação da Condição A:

Total de atendimentos com avaliação de Tuberculose = **34**.  

Pessoas beneficiadas com ao menos 1 atendimento = **11**.

Pessoas beneficiadas com ao menos 4 atendimentos = **2**.

In [12]:


df0 = lista_denominador
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

display(Markdown("### Perfil da Condição B"))

for P in indicador['condição_B'].keys():
    display(Markdown(f"{P}: {indicador['condição_B'][P]}  "))


### Perfil da Condição B

descrição: no mínimo 2 baciloscopias  

fonte: tb_fat_atend_individual  

profissional: enfermeiro  

procedimentos_solicitados: None  

procedimentos_avaliados: ['02.02.08.004-8', '02.02.08.006-4', '02.13.01.069-0']  

valor_min: 2  

In [13]:
display(Markdown("### Avaliação da Condição B:"))

df3 = pd.DataFrame()

for C in indicador['condição_B']['procedimentos_avaliados']:
    C = C.replace(".", "")
    C = C.replace("-", "")
    df1 = df[df['ds_filtro_proced_avaliados'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f"Procedimentos {C} avaliados **{N_atend}**  "))
    df3 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

procedimentos = df3.shape[0]
pessoas_beneficiadas = df3['paciente_id'].nunique()

df_pacientes_procedimentos = df3.groupby(['paciente_id']).agg(ds_filtro_proced_avaliados_count=('ds_filtro_proced_avaliados', 'count')).reset_index()
df_condição_B = df_pacientes_procedimentos[df_pacientes_procedimentos.ds_filtro_proced_avaliados_count >= int(indicador['condição_B']['valor_min'])]
pessoas_beneficiadas_B = df_condição_B.shape[0]

display(Markdown(f"Total de atendimentos com avaliação de baciloscopia = **{procedimentos}**.  "))
display(Markdown(f"Pessoas beneficiadas com ao menos 1 baciloscopia = **{pessoas_beneficiadas}**."))
display(Markdown(f"Pessoas beneficiadas com ao menos 2 baciloscopias = **{pessoas_beneficiadas_B}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Avaliação da Condição B:

Procedimentos 0202080048 avaliados **9**  

Procedimentos 0202080064 avaliados **2**  

Procedimentos 0213010690 avaliados **0**  

Total de atendimentos com avaliação de baciloscopia = **11**.  

Pessoas beneficiadas com ao menos 1 baciloscopia = **10**.

Pessoas beneficiadas com ao menos 2 baciloscopias = **1**.

In [14]:

display(Markdown("### Cálculo do indicador"))

display(Markdown(f"População-alvo: {populacao_alvo}"))

df_indicador = pd.merge(df_condição_A, df_condição_B, on='paciente_id')
# df_indicador = df_condição_A

pessoas_beneficiadas = df_indicador.shape[0]

Pessoas_não_atendidas = populacao_alvo - pessoas_beneficiadas

labels = ['Pessoas não atendidas', 'Pessoas beneficiadas']
values = [Pessoas_não_atendidas, pessoas_beneficiadas]
colors = ['blue', 'white']

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['red', 'blue']))
fig.show()

if populacao_alvo == 0:
    display(Markdown("### Não foi encontrada população-alvo. Não será possível calcular indicadores"))
    pessoas_beneficiadas = 0
else:
    indicador = pessoas_beneficiadas / populacao_alvo * 100
    indicador = locale.format_string('%10.2f', indicador)


if pessoas_beneficiadas == 0:
    display(Markdown("Nenhuma pessoa recebeu o tratamento preconizado neste indicador"))
else:
    display(Markdown(f"{indicador}% da população analisada recebeu o tratamento preconizado neste indicador."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Cálculo do indicador

População-alvo: 11

      9,09% da população analisada recebeu o tratamento preconizado neste indicador.

In [15]:
# df_indicador